In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pickle
#import keras

from sklearn import svm
import random
import os
from math import floor
import warnings

from sklearn.preprocessing import StandardScaler, MinMaxScaler


In [2]:
# If you want to train the model by yourself

# train data
with open('new_data_filter0.pickle', 'rb') as f:
    Data = pickle.load(f)
    Data = pd.DataFrame(Data)

numtrain = 186 * 6 * 12
numval = 2678


# Train validation split
idx = random.sample(range(numtrain), numval) # 80% for training
all_index = list(np.setdiff1d(range(numtrain), idx))

# Getting X_train, X_val, y_train and scale them

X_train = Data.iloc[all_index,:].drop(2251, axis = 1)
X_val = Data.iloc[idx,:].drop(2251, axis = 1)
y_train = pd.get_dummies(Data.iloc[all_index,:][2251])
#print(y_train)

names = y_train.columns
mapping = {}
i = 0
for n in names:
    mapping[i] = n
    i+=1
#print(mapping)

y_train = Data.iloc[all_index,:][2251]

scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

In [9]:
clf = svm.SVC(probability=True)  # class 
clf.fit(X_train, y_train)  # training the svc model

C:\Users\Hong\Anaconda3\envs\py36-tensorflow\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=True, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [59]:
Test = pd.read_csv('test_data_19.csv')

X_test = scaler.transform(Test.iloc[:,1:])
#pred = clf.predict(X_test)

probas = clf.predict_proba(X_test)

# 注意最右边是top1，最左边是老末。
后面有验证代码，可查阅！

In [60]:
top_n_predictions = np.argsort(probas, axis = 1)[:,-12:]

result = []
for pre in top_n_predictions:
    pre_list = []
    for index in pre:
        pre_list.append(mapping[index])
    result.append(pre_list)

    
for plist in result:
    print(plist)

['IDIWOH05', 'IDIWOH07', 'IDIWOH06', 'ZIF-9', 'ZIF-90', 'ZIF-67', 'ZIF-8', 'ZIF-12', 'ZIF-11', 'ZIF-71', 'JARMEU', 'JARMEU10']
['IDIWOH06', 'IDIWOH07', 'IDIWOH04', 'ZIF-9', 'ZIF-90', 'ZIF-67', 'ZIF-8', 'ZIF-12', 'ZIF-11', 'ZIF-71', 'JARMEU', 'JARMEU10']
['AFEHUO', 'HUYKIV', 'IDIWOH06', 'ZIF-9', 'ZIF-90', 'ZIF-67', 'ZIF-8', 'ZIF-12', 'ZIF-11', 'ZIF-71', 'JARMEU', 'JARMEU10']
['DUT-7', 'JARMEU', 'ZIF-7', 'ZIF-70', 'AFEHUO', 'QEFNAQ', 'DUT-12', 'ZIF-9', 'JARMEU10', 'ZIF-71', 'ZIF-10', 'DUT-10']
['ZIF-12', 'DUT-23', 'ZIF-70', 'AFEHUO', 'ZIF-7', 'DUT-10', 'JARMEU', 'DUT-12', 'QEFNAQ', 'ZIF-9', 'JARMEU10', 'ZIF-71']
['ZIF-12', 'DUT-23', 'ZIF-70', 'AFEHUO', 'ZIF-7', 'JARMEU', 'QEFNAQ', 'DUT-12', 'DUT-10', 'ZIF-9', 'JARMEU10', 'ZIF-71']
['IDIWOH03', 'IDIWOH05', 'IDIWOH07', 'AFIXES', 'ZIF-3', 'DUT-23', 'ZIF-67', 'MOF-5', 'ZIF-8', 'ZIF-65', 'ZIF-90', 'PIZJEN']
['IDIWOH05', 'JARMEU', 'JARMEU10', 'AFIXES', 'ZIF-3', 'DUT-23', 'ZIF-67', 'ZIF-8', 'MOF-5', 'ZIF-65', 'ZIF-90', 'PIZJEN']
['ZIF-71', 'ZIF

In [57]:
test_index1 = np.argsort(probas[0])[::-1][0:5]
test_value1 = np.sort(probas[0])[::-1][0:5]

print(test_index1)
print(test_value1)

[ 50  49 171 154 155]
[0.06499332 0.02786812 0.02467033 0.02356461 0.02231122]


In [58]:
test_index2 = np.argsort(probas[0], axis = 0)[-5:]
test_value2 = np.sort(probas[0], axis = 0)[-5:]

print(test_index2)
print(test_value2)

[155 154 171  49  50]
[0.02231122 0.02356461 0.02467033 0.02786812 0.06499332]
